In [3]:
# tools 
# agents 
# toolikit 
# wrappers above toolkit
#arXiv is an open archive for scholarly articles maintained and operated by Cornell University.
# we create and wikipedia and arxiv and create a wrappers

In [4]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

In [5]:
#doc_content_chars_max=200 This parameter defines the maximum number of characters to retrieve from the content of a Wikipedia article.
#top_k_results=1 This parameter specifies the number of top results to return from a Wikipedia search query. Setting it to 1 means that the wrapper will only return the most relevant result.

In [6]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [7]:
tool.name

'wikipedia'

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS 
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [10]:
retriever=vectordb.as_retriever()#Retriever will retrieve data from vector datasbase

In [11]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "search for information about Langsmith. For any question about langsmith you have to use this tool.")

In [12]:
retriever_tool.name

'langsmith_search'

In [13]:
## Arxiv tool
from langchain_community.utilities import  ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [14]:
tools=[tool,arxiv,retriever_tool] # first is wikipedia ,arxiv and retriever_tool

In [15]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Generative AI\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='search for information about Langsmith. For any question about langsmith you have to use this tool.', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000025CA0784540>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectors

In [16]:
## query from this tools  that is where we use agnets. Agents is responsible for taking which actions to take in responsible .
# what agents will do is it goes to wikipedia and if not arxiv


In [17]:
## Agents 
from langchain.agents import AgentType,initialize_agent,load_tools

In [25]:
# Creating a prompt # in langchain there is called a hub where people have alraedy created a prompt


from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [61]:
tools = [tool, arxiv, retriever_tool]

prompt = hub.pull("hwchase17/openai-functions-agent")

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    prompt=prompt, 
    verbose=True
)


In [62]:
response = agent({"input": "tell me about langsmith"})
print(response)




> Entering new AgentExecutor chain...


ValueError: Missing some input keys: {'page_content'}

In [63]:
print(prompt.messages)


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')]


In [70]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate,PromptTemplate,system_template

NameError: name 'system_template' is not defined

In [77]:
import os
from dotenv import load_dotenv
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
from langchain.agents import AgentType, initialize_agent
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate, PromptTemplate

# Load environment variables
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# Initialize LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

# Wikipedia Tool
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wikipedia_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

# Web Base Loader and Retriever Tool
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents, GoogleGenerativeAIEmbeddings(model="models/embedding-001"))
retriever = vectordb.as_retriever()
retriever_tool = create_retriever_tool(retriever, "langsmith_search", "search for information about Langsmith. For any question about langsmith you have to use this tool.")

# Arxiv Tool
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)

# Tools List
tools = [wikipedia_tool, arxiv_tool, retriever_tool]

# Define a custom prompt template
custom_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="You are a helpful assistant.")),
    HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template="{input}"))
])

# Agent Initialization
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    prompt=custom_prompt,
    verbose=True
)

# Define the correct input key structure
response = agent({"input": "tell me about langsmith","page_content":" "})
print(response)




> Entering new AgentExecutor chain...
Question: tell me about langsmith
Thought: I need to use the langsmith_search tool to find information about langsmith.
Action: langsmith_search
Action Input: "What is LangSmith" 
Observation: Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API key‚ÄãTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment‚ÄãShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though 

AttributeError: 'dict' object has no attribute 'output'